In [39]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import cv2
import numpy as np
import imutils
from keras.models import load_model
facedetect = cv2.CascadeClassifier(r'C:\Users\Abdelrahman Ragab\Downloads\Compressed\Project\haarcascade_frontalface_default.xml')
threshold=0.90
cap=cv2.VideoCapture(0)
#cap.set(3, 3024)
#cap.set(4, 4024)
font=cv2.FONT_HERSHEY_COMPLEX
from imutils.video import VideoStream
model = load_model(r'C:\Users\Abdelrahman Ragab\Downloads\Compressed\Project\FaceMaskDetection.h5')

def preprocessing(img):
    img=img.astype("uint8")
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img=cv2.equalizeHist(img)
    img=cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = img/255
    return img


def get_className(classNo):
    if classNo==0:
        return " Mask is Up"
    elif classNo==1:
        return " Warning No Mask!"

In [40]:
while True:
    image = np.empty((1,224,224,3))
    
    # Read the frame
    _, image = cap.read()
    image = imutils.resize(image, width=1024)
    
    faces = facedetect.detectMultiScale(image,1.3,5)
    for x,y,w,h in faces:
        crop_img=image[y:y+h,x:x+h]
        img=cv2.resize(crop_img, (224,224),3)
        img=preprocessing(img)
        img=img.reshape(1,224,224,3)
        
        prediction=model.predict(img)
        classIndex=np.argmax(prediction)
        probabilityValue=np.amax(prediction)
        if probabilityValue>threshold:
            if classIndex==0:
                cv2.rectangle(image,(x,y),(x+w,y+h),(0,240,0),2)
                cv2.rectangle(image, (x,y-40),(x+w, y), (0,240,0),-2)
                cv2.putText(image, str(get_className(classIndex)),(x,y-10), font, 0.75, (0,0,0),1, cv2.LINE_AA)
            elif classIndex==1:
                cv2.rectangle(image,(x,y),(x+w,y+h),(50,50,255),2)
                cv2.rectangle(image, (x,y-40),(x+w, y), (50,50,255),-2)
                cv2.putText(image, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)

    # cv2.putText(imgOrignal,str(round(probabilityValue*100, 2))+"%" ,(180, 75), font, 0.75, (255,0,0),2, cv2.LINE_AA)
    cv2.imshow("Result",image)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break


cap.release()
cv2.destroyAllWindows()